# Import

In [1]:
# data handling
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# time control
from time import sleep
from datetime import datetime

# NLP
import re
import ckonlpy
from ckonlpy.tag import Twitter
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.utils import simple_preprocess

# NLP Visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import wordcloud
from wordcloud import WordCloud
from PIL import Image
import cv2

# utils
from news_words import get_addwords, get_stopwords
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# 데이터 불러오기

In [2]:
# 오늘 날짜를 YYMMDD의 string형태로 저장
today = datetime.today().strftime("%y%m%d")
print(today)
df = pd.read_csv('./df_'+today+'.csv')
df

230627


,URL,신문사,기자명,제목,본문,날짜,연,월,일,요일
0,https://n.news.naver.com/mnews/article/277/000...,아시아경제,오수연,"KT클라우드, '클라우드 기반 NPU 인프라' 상용화",초거대 인공지능(AI) 연구?개발을 위한 AI 반도체 인프라를 온라인 전용 포털에서...,2023-06-27,2023,6,27,1
1,https://n.news.naver.com/mnews/article/016/000...,헤럴드경제,김현일,네이버·카카오로도 SRT 승차권 예매·자동차 검사 예약한다,네이버 지도 앱(왼쪽)과 카카오 T 앱에서 SRT 예매하는 화면. [디지털플랫폼정부...,2023-06-26,2023,6,26,0
2,https://n.news.naver.com/mnews/article/018/000...,이데일리,함정선,"SKT, 청년요금제 인기…신규·변경 고객 10명 중 7명 '선택'",만 19~34세 5G로 변경 청년 고객 73% 가입\n연말까지 가입자 100만명 목...,2023-06-27,2023,6,27,1
3,https://n.news.naver.com/mnews/article/008/000...,머니투데이,류준영,"박윤규 제2차관, '미래 유니콘' 15개 스타트업에 인증서 수여",'글로벌 ICT 미래 유니콘 육성사업' 선정 15개사 인증서 수여\n박윤규 과학기술...,2023-06-27,2023,6,27,1
4,https://n.news.naver.com/mnews/article/030/000...,전자신문,정보없음,"한글과컴퓨터, 포스코DX와 MOU 체결...“SDK 사업 확대 나선다”",한글과컴퓨터가 포스코DX와 인공지능(AI)기반 소프트웨어개발키트(SDK) 사업 확대...,2023-06-27,2023,6,27,1
...,...,...,...,...,...,...,...,...,...,...
203,https://n.news.naver.com/mnews/article/009/000...,매일경제,정보없음,"컴투스홀딩스 야심작 ‘제노니아’, 국내 시장 흥행 ‘정조준’",27일 0시 정식 서비스 돌입…사전예약 200만 ‘기대감’\n\n\n컴투스홀딩스의 ...,2023-06-26,2023,6,26,0
204,https://n.news.naver.com/mnews/article/008/000...,머니투데이,정보없음,"디도스에 당한 디아블로, 12시간 '먹통'에 이용자들 폭발",[서울=뉴시스] 권창회 기자 = 디아블로4가 출시 5일 만에 전 세계 6억 6600...,2023-06-26,2023,6,26,0
205,https://n.news.naver.com/mnews/article/092/000...,ZDNet Korea,조성진,주요국 고강도 통화정책 행보…韓 7월 금통위 향방은?,"이창용 총재, '금리인상' 가능성 시사…시장선 '동결' 점쳐\n주요국 중앙은행이 고...",2023-06-26,2023,6,26,0
206,https://n.news.naver.com/mnews/article/366/000...,조선비즈,이종현,“양자기술은 산업의 게임 체인저”...전세계 양자 전문가·기업들 서울에 모였다,26일 국내 최대 양자기술 행사 ‘퀀텀 코리아 2023′ 개막\n\n26일 오후 서...,2023-06-26,2023,6,26,0


In [3]:
# 유영 스크랩 csv를 합친다면
df = pd.concat([df, pd.read_csv('./20230626_20230627_news.csv')])
df = df.drop_duplicates(['제목'])
df = df.reset_index(drop=True)
df

,URL,신문사,기자명,제목,본문,날짜,연,월,일,요일
0,https://n.news.naver.com/mnews/article/277/000...,아시아경제,오수연,"KT클라우드, '클라우드 기반 NPU 인프라' 상용화",초거대 인공지능(AI) 연구?개발을 위한 AI 반도체 인프라를 온라인 전용 포털에서...,2023-06-27,2023,6,27,1
1,https://n.news.naver.com/mnews/article/016/000...,헤럴드경제,김현일,네이버·카카오로도 SRT 승차권 예매·자동차 검사 예약한다,네이버 지도 앱(왼쪽)과 카카오 T 앱에서 SRT 예매하는 화면. [디지털플랫폼정부...,2023-06-26,2023,6,26,0
2,https://n.news.naver.com/mnews/article/018/000...,이데일리,함정선,"SKT, 청년요금제 인기…신규·변경 고객 10명 중 7명 '선택'",만 19~34세 5G로 변경 청년 고객 73% 가입\n연말까지 가입자 100만명 목...,2023-06-27,2023,6,27,1
3,https://n.news.naver.com/mnews/article/008/000...,머니투데이,류준영,"박윤규 제2차관, '미래 유니콘' 15개 스타트업에 인증서 수여",'글로벌 ICT 미래 유니콘 육성사업' 선정 15개사 인증서 수여\n박윤규 과학기술...,2023-06-27,2023,6,27,1
4,https://n.news.naver.com/mnews/article/030/000...,전자신문,정보없음,"한글과컴퓨터, 포스코DX와 MOU 체결...“SDK 사업 확대 나선다”",한글과컴퓨터가 포스코DX와 인공지능(AI)기반 소프트웨어개발키트(SDK) 사업 확대...,2023-06-27,2023,6,27,1
...,...,...,...,...,...,...,...,...,...,...
1033,https://v.daum.net/v/20230627043302905,한국경제,최진석,"구글 전 CEO 에릭 슈밋의 경고…""AI 거짓정보로 내년 미국 대선 혼탁해질 것""",사진=한경DB\n에릭 슈밋 구글 전 최고경영자(CEO)가 생성 인공지능(AI)에 의...,2023-06-27,2023,6,27,1
1034,https://v.daum.net/v/20230627041002837,한국경제,최진석,소프트웨어 회사로 변모하는 IBM…46억달러에 앱티오 인수,IBM이 소프트웨어 개발사 앱티오를 46억달러(6조원)에 인수한다고 26일(현지시간...,2023-06-27,2023,6,27,1
1035,https://v.daum.net/v/20230627024002378,한국경제,최진석,‘메타버스 대중화’ 돌파구 찾는 메타…'VR 구독 서비스' 출시,"월 7.99달러 내면 매월 새로운 게임 2개 제공\n메타 VR 기기, 퀘스트 충성고...",2023-06-27,2023,6,27,1
1036,https://v.daum.net/v/20230627011954198,중앙일보,입력,[사진] 국내 첫 로봇 지휘자 “내 손을 보세요”,국내 첫 로봇 지휘자 “내 손을 보세요”\n지휘하는 로봇 ‘에버(EveR)6’가 2...,2023-06-27,2023,6,27,1


# 전처리

In [4]:
# raw text와 processed text를 비교하기 위해 copy
df_preprocess = df.copy()
df_preprocess['본문'] = df_preprocess['본문'].str.upper() # 전부 대문자로 변경

# twitter 객체에 Noun 단어 adding
twitter = Twitter()

# 사용자 정의 명사 지정 리스트 받기
add_words = get_addwords()
# 불용어 리스트 받기
stop_words = get_stopwords()

for noun in add_words:
    twitter.add_dictionary(noun, 'Noun')

# re와 ckonlpy로 전처리
articles = df_preprocess['본문']

# 본문 Seriese객체에서 index 받아오기
# 기사의 건수(약 1400건)만큼 iter
for idx in tqdm(range(len(articles))): 
    # re로 기본적인 전처리
    text = articles[idx]
    to_remove = '[\n,@\'()‘“”’%./■△\"·]+'
    text = re.sub(to_remove, '', text)
    
    # ckonlpy로 ' '로 구분된 string형태의 corpus 생성
    contents =''
    for temp_word in twitter.nouns(text):
        # 한글자와 불용어 제외
        if (len(temp_word) > 1)&(temp_word not in stop_words):
            contents = contents + ' ' + temp_word
    
    # 전처리를 거친 corpus를 df_preprocess의 본문에 저장
    df_preprocess.loc[idx, '본문'] = contents.strip()

######################################## Check result
sleep(2)
print("전처리 전 :")
print("*"*50)
display(df.head(3))
display(df.tail(3))
print('\n\n')
print("전처리 후 :")
print("*"*50)
display(df_preprocess.head(3))
display(df_preprocess.tail(3))
########################################

100%|██████████| 1038/1038 [00:31<00:00, 32.85it/s]


전처리 전 :
**************************************************


,URL,신문사,기자명,제목,본문,날짜,연,월,일,요일
0,https://n.news.naver.com/mnews/article/277/000...,아시아경제,오수연,"KT클라우드, '클라우드 기반 NPU 인프라' 상용화",초거대 인공지능(AI) 연구?개발을 위한 AI 반도체 인프라를 온라인 전용 포털에서...,2023-06-27,2023,6,27,1
1,https://n.news.naver.com/mnews/article/016/000...,헤럴드경제,김현일,네이버·카카오로도 SRT 승차권 예매·자동차 검사 예약한다,네이버 지도 앱(왼쪽)과 카카오 T 앱에서 SRT 예매하는 화면. [디지털플랫폼정부...,2023-06-26,2023,6,26,0
2,https://n.news.naver.com/mnews/article/018/000...,이데일리,함정선,"SKT, 청년요금제 인기…신규·변경 고객 10명 중 7명 '선택'",만 19~34세 5G로 변경 청년 고객 73% 가입\n연말까지 가입자 100만명 목...,2023-06-27,2023,6,27,1


,URL,신문사,기자명,제목,본문,날짜,연,월,일,요일
1035,https://v.daum.net/v/20230627024002378,한국경제,최진석,‘메타버스 대중화’ 돌파구 찾는 메타…'VR 구독 서비스' 출시,"월 7.99달러 내면 매월 새로운 게임 2개 제공\n메타 VR 기기, 퀘스트 충성고...",2023-06-27,2023,6,27,1
1036,https://v.daum.net/v/20230627011954198,중앙일보,입력,[사진] 국내 첫 로봇 지휘자 “내 손을 보세요”,국내 첫 로봇 지휘자 “내 손을 보세요”\n지휘하는 로봇 ‘에버(EveR)6’가 2...,2023-06-27,2023,6,27,1
1037,https://v.daum.net/v/20230627000250726,중앙일보,입력,"[사진] 파티에 제격, 포터블 스피커","파티에 제격, 포터블 스피커\nLG전자가 내장 배터리를 탑재한 포터블 파티 스피커 ...",2023-06-27,2023,6,27,1





전처리 후 :
**************************************************


,URL,신문사,기자명,제목,본문,날짜,연,월,일,요일
0,https://n.news.naver.com/mnews/article/277/000...,아시아경제,오수연,"KT클라우드, '클라우드 기반 NPU 인프라' 상용화",거대 인공지능 AI 연구 개발 AI 반도체 인프라 온라인 전용 포털 클릭 번만 이용...,2023-06-27,2023,6,27,1
1,https://n.news.naver.com/mnews/article/016/000...,헤럴드경제,김현일,네이버·카카오로도 SRT 승차권 예매·자동차 검사 예약한다,네이버 지도 왼쪽 카카오 매하 화면 디지털 플랫폼 정부위원 헤럴드경제 김현 일부 네...,2023-06-26,2023,6,26,0
2,https://n.news.naver.com/mnews/article/018/000...,이데일리,함정선,"SKT, 청년요금제 인기…신규·변경 고객 10명 중 7명 '선택'",5G 변경 청년 고객 가입 연말 가입자 목표 이데일리 함정 SK 텔레콤 이하 5G ...,2023-06-27,2023,6,27,1


,URL,신문사,기자명,제목,본문,날짜,연,월,일,요일
1035,https://v.daum.net/v/20230627024002378,한국경제,최진석,‘메타버스 대중화’ 돌파구 찾는 메타…'VR 구독 서비스' 출시,달러 내면 매월 게임 메타 VR 기기 퀘스트 충성 고객 확대 겨냥 메타버스 대중화 ...,2023-06-27,2023,6,27,1
1036,https://v.daum.net/v/20230627011954198,중앙일보,입력,[사진] 국내 첫 로봇 지휘자 “내 손을 보세요”,국내 로봇 지휘자 요지 휘하 로봇 에버 오후 서울 중구 국립극장 오름 극장 다흐 빌...,2023-06-27,2023,6,27,1
1037,https://v.daum.net/v/20230627000250726,중앙일보,입력,"[사진] 파티에 제격, 포터블 스피커",파티 제격 포터블 스피커 LG전자 내장 배터리 탑재 포터블 파티 스피커 LG 엑스 ...,2023-06-27,2023,6,27,1


In [5]:
articles = df_preprocess['본문'].tolist()
articles[0]

'거대 인공지능 AI 연구 개발 AI 반도체 인프라 온라인 전용 포털 클릭 번만 이용 KT 클라우드 토종 팹리스 반도체 설계 전문 아톰 적용 클라우드 기반 신경망 처리장치 인프라 상용 일부 개시 다리 아톰 적용 KT 인프라 KT 클라우드 AI 최적화 설계 AI 반도체 린다 그래픽 처리장치 GPU 대비 연산 속도 전력 소모 AI 연구개발 기간 비용 혁신 KT 클라우드 협력 글로벌 고성능 인정 아톰 탑재 클라우드 기반 인프라 상용 KT 클라우드 클라우드 기반 이용 플랫폼 적용 자원 공유 구성 관리 오토 프로비저닝 구현 전용 포털 간편 기반 AI 학습 추론 진행 클라우드 기반 인프라 클릭 고객 직접 자원 성하 연산 세션 활용 관리 모니터링 구축 인프라 대비 사용 의성 불어 고성능 전력 특성 필요 필요 만큼 이용 환경 AI 연구 기간 비용 혁신 KT 클라우드 인프라 AI 반도체 초기 수요 중소 벤처기업 대상 국산 AI 반도체 기반 컴퓨팅 자원 목적 클라우드 프로젝트 사업 AI 사업 고성능 컴퓨팅 사업 대상 먼저 올해 하반기 고객 대상 확대 계획 탑재 아톰 금융 거래 특화된 AI 반도체 아이온 보이 AI 반도체 국내 팹리스 개발 중인 세대 수준 성능 상태 아톰 트랜스포머 언어 모델 기계 번역 모델 부동 소수점 연산 유일 국산 AI 반도체 검증 대회 엠엘퍼프 결과 아톰 엔비디아 퀄컴 클라우드 AI 대비 상처리 언어 모델 AR 확인 최대 와트 전력 설계 삼성 나노 자외선 EU 공정 제조 업계 주로 사용 중인 GPU 교하면 모델 언어모델 전력 효율 전력 소모 기적 다리 아톰 성능 업그레이드 버전 준비 중이 KT 클라우드 인프라 고도화 이후 차기 모델 리벨 계획 박성현 인프라 대해 최근 엠엘퍼프 전과 언어 모델 모두 글로벌 티어 수준 역량 입증 아톰 KT 클라우드 데이터 센터 상용 자부심 토대 대한민국 AI 산업 활성화 기여 AI 반도체 리딩 KT 클라우드 향후 AR 클라우드 플랫폼 설계 구축 다양 AI 응용 AI 반도체 레퍼런스 확보 계획 다양 협력 비용 고성능 효율 AI 

# 토픽 모델링 및 Topic Num 최적화

https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/

In [6]:
# 리스트 내 리스트 형태로 저장
preprocessed_articles = [article.split(' ') for article in articles]

In [7]:
# corpora.Dictionary
dictionary = corpora.Dictionary(preprocessed_articles)
dictionary.filter_extremes(no_below=20, no_above=0.6)
# 빈도수가 

corpus = [dictionary.doc2bow(article) for article in preprocessed_articles]
texts = [preprocessed_articles[idx] for idx in range(len(corpus))]

**하이퍼 파라미터 세팅**

In [8]:
PARAM = {'RANGES':20,
         'PASSES':20,
         'ITER':500,
         'COHERENCE_METRIC':'c_v',
}

In [ ]:
# 2부터 20까지의 토픽 수를 비교하며 정합도 점수 기록
news_coherence_scores = []

for i in tqdm(range(3, PARAM['RANGES']+1), desc="Operating All Topic Ranges"):
    model = LdaModel(corpus=corpus, num_topics=i, id2word=dictionary, passes=PARAM['PASSES'], iterations=PARAM['ITER'])
    coherence_model = CoherenceModel(model=model, texts=texts, corpus=corpus, dictionary=dictionary, coherence=PARAM['COHERENCE_METRIC'])
    coherence_lda = coherence_model.get_coherence()
    news_coherence_scores.append(coherence_lda)

Operating All Topic Ranges:  78%|███████▊  | 14/18 [08:39<02:24, 36.13s/it]

In [ ]:
k = [i for i in range(3, PARAM['RANGES']+1)]
x = np.array(k)
y = np.array(news_coherence_scores)
plt.figure(figsize=(10, 5))
plt.plot(x, y)
plt.xlabel("Number Of Topic(2-"+str(PARAM['RANGES'])+")")
plt.ylabel('Coherence Score')
plt.show()

In [ ]:
news_coherence_scores

# Topic Num이 최적화된 모델

In [ ]:
find_best = np.array([news_coherence_scores])
Best_Num_Of_Topic = int(np.where(y==find_best.max())[0][0]+3)
print("최적의 토픽 수 :", Best_Num_Of_Topic)
print("정합도 점수 :", round(find_best.max(), 4))

In [ ]:
num_topics = Best_Num_Of_Topic
lda_model = LdaModel(corpus, num_topics, dictionary, passes=PARAM['PASSES'], iterations=PARAM['ITER'])

coherence_model = CoherenceModel(model=lda_model, texts=texts, corpus=corpus, dictionary=dictionary)

vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

# 각 토픽에 정합하는 기사 추출

Note : preprocessed_articles 를 토큰화 한 것이 corpus

In [ ]:
# 단계 1 : 각 문서에 대한 주제 분포 가져오기
document_topics = [lda_model.get_document_topics(doc) for doc in corpus]
# document_topics # 1382개

main_topic_list = []

for article_idx, article in enumerate(document_topics):
    max_score = 1e-8
    max_topic_num = -1e-8
    
    # 한 기사안에서 토픽 : 점수
    for topicnum, score in article:
        if score > max_score:
            max_score = score
            max_topic_num = topicnum
    #main_topic_list.append((str(max_topic_num+1)+'번토픽', max_score))
    main_topic_list.append((int(max_topic_num)+1, max_score))

In [ ]:
# 토픽과 점수 추가하여 df_merged로 합치기
df_score = pd.DataFrame(main_topic_list, columns=['토픽', '점수'])
df_merged = pd.concat([df_preprocess, df_score], axis=1)

In [ ]:
# 높은 점수 순서대로 정렬한 후, 토픽으로 group을 생성한 후, 상위 5개씩 추출
df_merged = df_merged.sort_values('점수', ascending=False)
df_result = df_merged.groupby('토픽').head(5)
df_result = df_result.sort_values('토픽', ignore_index=True)

In [ ]:
# 최종 기사 저장
today = datetime.today().strftime("%y%m%d")
df_result.drop(columns=['본문']).to_csv('result_'+today+'.csv', header=False)

In [ ]:
# 최대 행 제한 해제
pd.set_option('display.max_rows', None)
# 최대 열 제한 해제
pd.set_option('display.max_columns', None)

# 결과 확인
df_result

# 토픽 당 워드클라우드

In [ ]:
df_result

In [ ]:
def create_wordcloud(df_result=df_result, topic_num=None, width=600, height=600, max_words=30):
    mask = cv2.imread('./circle.png')
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = cv2.resize(mask, (width, height))
    
    text = ' '.join(df_result[df_result['토픽']==topic_num]['본문'])
    wordcloud = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf', width=width, height=height, max_words=max_words,
                          background_color='white', collocations=False, prefer_horizontal=1, mask=mask).generate(text)
    plt.figure(figsize=(width/100, height/100))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
create_wordcloud(topic_num=17)